In [1]:
import numpy as np
import logging 
#3D viewer from js
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.systems.analysis import Simulator 
from pydrake.systems.framework import DiagramBuilder

from manipulation import running_as_notebook
from manipulation.meshcat_utils import MeshcatPoseSliders, WsgButton 
from manipulation.scenarios import (AddIiwaDifferentialIK, 
                                   MakeManipulationStation)

In [2]:
meshcat = StartMeshcat() #from pydrake.geometry

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
meshcat.port()

7000

In [4]:
model_directives = """
directives:
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/urdf/planar_iiwa14_spheres_dense_elbow_collision.urdf
    default_joint_positions:
        iiwa_joint_2: [0.1]
        iiwa_joint_4: [-1.2]
        iiwa_joint_6: [1.6]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
- add_model:
    name: foam_brick
    file: package://drake/examples/manipulation_station/models/061_foam_brick.sdf
    default_free_body_pose:
        base_link:
            translation: [0.6, 0, 0]
- add_model:
    name: robot_table
    file: package://drake/examples/kuka_iiwa_arm/models/table/extra_heavy_duty_table_surface_only_collision.sdf
- add_weld:
    parent: world
    child: robot_table::link
    X_PC:
        translation: [0, 0, -0.7645]
- add_model:
    name: work_table
    file: package://drake/examples/kuka_iiwa_arm/models/table/extra_heavy_duty_table_surface_only_collision.sdf
- add_weld:
    parent: world
    child: work_table::link
    X_PC:
        translation: [0.75, 0, -0.7645]
"""

In [10]:
builder = DiagramBuilder()
station = builder.AddSystem(MakeManipulationStation(model_directives, time_step=0.0001))


In [11]:
plant = station.GetSubsystemByName('plant')

In [13]:
controller_plant = station.GetSubsystemByName('iiwa_controller').get_multibody_plant_for_control()
controller_plant

<pydrake.multibody.plant.MultibodyPlant_[float] at 0x7f86c9bcebb0>

In [15]:
visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat
    )
meshcat.Set2dRenderMode(xmin=-0.25, xmax=1.5, ymin=-0.1, ymax=1.3)
meshcat.DeleteAddedControls()

In [16]:
def teleop_2d():
    builder = DiagramBuilder() #system.framework
    
    time_step = 0.001
    station = builder.AddSystem(
        MakeManipulationStation(model_directives, time_step=time_step)
    )
    plant = station.GetSubsystemByName('plant')
    controller_plant = station.GetSubsystemByName(
        "iiwa_controller"
    ).get_multibody_plant_for_control()
    
    
    # still from pydrake
    # rendering part: 
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat
    )
    meshcat.Set2dRenderMode(xmin=-0.25, xmax=1.5, ymin=-0.1, ymax=1.3)
    meshcat.DeleteAddedControls()
    
    # Differential Inverse Kinematics part ---> q' = J^-1 * r' : 
    # addIiwaDifferentialIK comes from manipulation 
    differential_ik = AddIiwaDifferentialIK(
        builder, 
        controller_plant, 
        frame=controller_plant.GetFrameByName('iiwa_link_7')
    )
    # connect system ---> note that it's done by port
    builder.Connect(
        differential_ik.get_output_port(), 
        station.GetInputPort("iiwa_position")
    )
    builder.Connect(
        station.GetOutputPort("iiwa_state_estimated"),
        differential_ik.GetInputPort("robot_state")
    )
    
    
    # Teleop widgets: 
    teleop = builder.AddSystem(
        
        MeshcatPoseSliders(
            meshcat, 
            min_range=MeshcatPoseSliders.MinRange(roll=0, x=-0.6, z=0.0),
            max_range=MeshcatPoseSliders.MaxRange(roll=2 * np.pi, x=0.8, z=1.1),
            value=MeshcatPoseSliders.Value(pitch=0, yaw=np.pi / 2, y=0),
            visible=MeshcatPoseSliders.Visible(pitch=False, yaw=False, y=False),
            decrement_keycode=MeshcatPoseSliders.DecrementKey(
                x="ArrowLeft",
                z="ArrowDown"),
            increment_keycode=MeshcatPoseSliders.IncrementKey(
                x="ArrowRight",
                z="ArrowUp"),
            body_index=plant.GetBodyByName("iiwa_link_7").index()
        )
    )
    
    builder.Connect(
        teleop.get_output_port(0),
        differential_ik.get_input_port(0)
    )
    builder.Connect(
        station.GetOutputPort('body_poses'), 
        teleop.GetInputPort('body_poses')
    )
    # from manipulation 
    wsg_teleop = builder.AddSystem(WsgButton(meshcat))
    builder.Connect(wsg_teleop.get_output_port(0), 
                   station.GetInputPort("wsg_position"))
    
    diagram = builder.Build()
    simulator = Simulator(diagram)
    
    
    
    
    if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.set_target_realtime_rate(1.0)
        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
        meshcat.DeleteButton("Stop Simulation")
    else:
        simulator.AdvanceTo(0.1)
    


In [18]:
def teleop_2d_with_return_diagram():
    builder = DiagramBuilder() #system.framework
    
    time_step = 0.001
    station = builder.AddSystem(
        MakeManipulationStation(model_directives, time_step=time_step)
    )
    plant = station.GetSubsystemByName('plant')
    controller_plant = station.GetSubsystemByName(
        "iiwa_controller"
    ).get_multibody_plant_for_control()
    
    
    # still from pydrake
    # rendering part: 
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat
    )
    meshcat.Set2dRenderMode(xmin=-0.25, xmax=1.5, ymin=-0.1, ymax=1.3)
    meshcat.DeleteAddedControls()
    
    # Differential Inverse Kinematics part ---> q' = J^-1 * r' : 
    # addIiwaDifferentialIK comes from manipulation 
    differential_ik = AddIiwaDifferentialIK(
        builder, 
        controller_plant, 
        frame=controller_plant.GetFrameByName('iiwa_link_7')
    )
    # connect system ---> note that it's done by port
    builder.Connect(
        differential_ik.get_output_port(), 
        station.GetInputPort("iiwa_position")
    )
    builder.Connect(
        station.GetOutputPort("iiwa_state_estimated"),
        differential_ik.GetInputPort("robot_state")
    )
    
    
    # Teleop widgets: 
    teleop = builder.AddSystem(
        
        MeshcatPoseSliders(
            meshcat, 
            min_range=MeshcatPoseSliders.MinRange(roll=0, x=-0.6, z=0.0),
            max_range=MeshcatPoseSliders.MaxRange(roll=2 * np.pi, x=0.8, z=1.1),
            value=MeshcatPoseSliders.Value(pitch=0, yaw=np.pi / 2, y=0),
            visible=MeshcatPoseSliders.Visible(pitch=False, yaw=False, y=False),
            decrement_keycode=MeshcatPoseSliders.DecrementKey(
                x="ArrowLeft",
                z="ArrowDown"),
            increment_keycode=MeshcatPoseSliders.IncrementKey(
                x="ArrowRight",
                z="ArrowUp"),
            body_index=plant.GetBodyByName("iiwa_link_7").index()
        )
    )
    
    builder.Connect(
        teleop.get_output_port(0),
        differential_ik.get_input_port(0)
    )
    builder.Connect(
        station.GetOutputPort('body_poses'), 
        teleop.GetInputPort('body_poses')
    )
    # from manipulation 
    wsg_teleop = builder.AddSystem(WsgButton(meshcat))
    builder.Connect(wsg_teleop.get_output_port(0), 
                   station.GetInputPort("wsg_position"))
    
    diagram = builder.Build()
    return diagram 
    
    
    
    
    if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.set_target_realtime_rate(1.0)
        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
        meshcat.DeleteButton("Stop Simulation")
    else:
        simulator.AdvanceTo(0.1)

In [17]:
teleop_2d()

Keyboard Controls:
roll : KeyQ / KeyE
x : ArrowLeft / ArrowRight
z : ArrowDown / ArrowUp
Press Space to open/close the gripper
Press Escape to stop the simulation


In [19]:
diagram = teleop_2d_with_return_diagram()

Keyboard Controls:
roll : KeyQ / KeyE
x : ArrowLeft / ArrowRight
z : ArrowDown / ArrowUp
Press Space to open/close the gripper


In [25]:
import pydot
from IPython.display import SVG

graph  = pydot.graph_from_dot_data(diagram.GetGraphvizString())[0]

In [26]:
graph.write_png('station_struct.png')